# Sumarização de textos
## Processamento de Linguagem Natural
Nesta aula trabalharemos com uma tarefa de PLN muito popular, a Sumarização de textos. O objetivo é que ao final desta aula você entenda as [principais etapas](https://drive.google.com/file/d/10nLM6wmsXcsSmqj_dskNU7pDHBXAuSyu/view?usp=sharing) de um algoritmo de sumarização e saiba aplicar um método simples.

Existem diversas bibliotecas que fornecem implementações de diversos algoritmos de sumarização prontos, como o Gensim e Sumy. Porém para entender melhor cada etapa, iremos implementar um algoritmo simples.

## Exemplo de sumarização baseada em frequência
*   Adaptação por Lucas Oliveira
*   Algoritmo original por Akash Panchal e disponível em https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65

### Importa os módulos de NLP do NLTK

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
import operator

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


### Função que cria a tabela de frequência das palavras
Todas palavras passam pelo algoritmo de Stemming e as stopwords são retiradas.
Então as frequências das palavras são calculadas.

In [ ]:
def tabela_de_frequencia(text_string) -> dict:

    # Assim como em toda tarefa de PLN, é muito importante o pré-processamento
    # Retiramos aqui apenas as stopwords e pontuação, mas dependendo do contexto
    # da sumarização, talvez fosse necessária a retirada de outros itens
    # Ex.: Nomes de pessoas em uma descrição de chat, nomes de seções em documentos, etc
    stopWords = set(stopwords.words("portuguese") + list(punctuation))
    words = word_tokenize(text_string)
    stemmer = nltk.stem.RSLPStemmer()

    freqTable = dict()
    for word in words:
      #word = stemmer.stem(word)
      #if word in stopWords:
        #continue
        # Se palavra já está na tabela
        if word in freqTable:
            # Incrementa sua frequencia
            freqTable[word] += 1
        else:
            # Cria posição da palavra na tabela
            freqTable[word] = 1

    return freqTable

### Função que define a pontuação das sentenças
Para cada sentença calcula-se a pontuação baseado nas palavras da sentença que estão presentes na tabela de frequências.
Exemplo:
```
freqTable = {'teste':5, 'sumarização':2}
sentence = "Este é um teste de sumarização"
```
Para a sentença acima a pontuação será 7, pois a palavra teste tem valor 5 e a palavra sumarização 2.

---

Ao final a pontuação da sentença é divida pela quantidade de palavras da sentença, para evitar que sentenças longas levem vantagem sobre sentenças curtas.



In [ ]:
def pontuar_sentencas(sentences, freqTable) -> dict:

    sentenceValue = dict()

    # Percorre as sentenças
    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        word_count_in_sentence_except_stop_words = 0
        # Percorre as palavras na tabela de frequência
        for wordValue in freqTable:
            # Caso encontre a palavra na sentença atual
            if wordValue in sentence.lower():
                # Incrementa contador
                word_count_in_sentence_except_stop_words += 1
                # Se a sentença atual já tem pontuação atribuida
                if sentence[:10] in sentenceValue:
                    # Soma nova pontuação
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    # Atribui valor da palavra atual para a sentença
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        # Se a sentença atual teve pontuação atribuida
        if sentence[:10] in sentenceValue:
            # Dividimos sua pontuação pela quantidade de palavras da sentença - para evitar que sentenças longas levem vantagem sobre sentenças curtas.
            sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] / word_count_in_sentence_except_stop_words

    return sentenceValue

### Função que calcula a pontuação média de todas as sentenças
Calcula a pontuação média de todas as sentenças

In [ ]:
def calcula_pontuacao_media(pontuacao_sentencas) -> int:

    sumValues = 0
    for entry in pontuacao_sentencas:
        sumValues += pontuacao_sentencas[entry]

    average = (sumValues / len(pontuacao_sentencas))

    return average

### Funções que geram o sumário de acordo com pontuação com número de sentenças ou taxa de compressão

In [ ]:
def gerar_sumario(sentencasOriginais, pontuacao_sentencas, threshold):
    contador_sentencas = 0
    sumario = ''

    for sentencaOriginal in sentencasOriginais:
        #Threshold de pontuação
        if sentencaOriginal[:10] in pontuacao_sentencas and pontuacao_sentencas[sentencaOriginal[:10]] >= (threshold):

            sumario += " " + sentencaOriginal
            contador_sentencas += 1

    return sumario


def gerar_sumario_fixo(sentencasOriginais, pontuacao_sentencas, threshold):
    contador_sentencas = 0
    sumario = ''

    # Desenvolva a lógica aqui

    return sumario

def gerar_sumario_compressao(sentencasOriginais, pontuacao_sentencas, threshold):
    contador_sentencas = 0
    sumario = ''

    # Desenvolva a lógica aqui

    return sumario

### Exemplo de textos de entrada

In [ ]:
# Texto 1
text = """Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa.

Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero.

O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”.

Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México).

Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos. O ataque deixou seis mortos.

Os confessores, identificados como Particio Reyes, Jonathan Osorio e Agustín García Reyes, dizem que receberam os 43 estudantes no lixão de Cocula, a 22 km de Iguala. Segundo os pistoleiros, 15 deles chegaram ao local mortos com sinais de asfixia.

Segundo a Procuradoria-Geral do México, os detidos não disseram quem levou os estudantes e quem era o mandante da emboscada.

O órgão, porém, acredita que o mandante foi o prefeito de Iguala, Jose Luis Abarca, preso na quarta (5).

A intenção seria evitar que os alunos atrapalhassem um evento em que sua mulher, María de los Ángeles Pineda, seria lançada como candidata a sucedê-lo. A mulher de Abarca é irmã de três chefes do Guerreros Unidos.

FAMILIARES

Em entrevista, os parentes disseram não acreditar na versão do procurador-geral e pediram que o material recolhido seja analisado por peritos independentes.

Para eles, o governo quer fazer com que eles acreditem que seus filhos estão mortos. “Sequer mostraram fotos dos nossos filhos. Enquanto não houver provas, nossos filhos estão vivos”, disse Felipe de la Cruz, pai de um dos alunos.

Os pais pediram ao governo que prossiga com as buscas e permita a assistência técnica da Comissão Interamericana de Direitos Humanos."""



In [ ]:
# Texto 2
text = '''O ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF), validou nesta terça-feira (17) o acordo para utilização do fundo bilionário da Petrobras – de cerca de R$ 2,6 bilhões.

O acordo foi assinado pelo governo federal, representantes de Câmara e Senado e da Procuradoria Geral da República (PGR) e enviado para homologação pelo ministro. Moraes validou a seguinte divisão para uso do fundo:

1) R$ 1,601 bilhão para educação, sendo:

R$ 1 bilhão para ações relacionadas à educação infantil;
R$ 250 milhões para o Ministério da Cidadania, para ações relacionadas ao Programa Criança Feliz (desenvolvimento integral da Primeira Infância);
R$ 250 milhões para o Ministério da Ciência, Tecnologia, Inovações e Comunicações, para ações de empreendedorismo e bolsas do Conselho Nacional de Desenvolvimento Científico e Tecnológico (CNPq);
R$ 100 milhões para ações socioeducativas em cooperação com os Estados, preferencialmente por intermédio do Ministério da Mulher, da Família e dos Direitos Humanos.
2) R$ 1,060 bilhão para Amazônia (prevenção, fiscalização e combate a desmatamentos e incêndios), sendo:

R$ 630 milhões para administração pela União de ações de operações de Garantia da Lei e da Ordem (GLO) e outros institutos;
R$ 430 milhões para execução pelos estados da região amazônica.
Na decisão, o ministro determinou a imediata transferência dos recursos financeiros depositados, devidamente corrigidos, para uma conta única do Tesouro Nacional para que o acordo seja cumprido integralmente.

Com relação ao uso R$ 430 milhões na Amazônia legal, Moraes afirmou que a utilização dos recursos deve observar:

área territorial do Estado;
população estimada na data da homologação do acordo;
o inverso do PIB per capita dos Estados, o número de focos de queimadas;
a área desmatada total por Estado.
O ministro também concedeu pedido da Caixa para que a instituição desconte a remuneração relativa ao tempo em que permaneceu com a guarda dos valores.


Alexandre de Moraes declarou ainda a nulidade do acordo firmado entre a força-tarefa da Lava Jato e a Petrobras. O ministro já havia suspenso a eficácia do acordo em março.

"A eventual apropriação, por determinados membros do Ministério Público, da administração e destinação de proveito econômico resultante da atuação do órgão, além de desrespeitar os princípios da legalidade, da impessoalidade e da moralidade administrativa, implicou séria agressão ao perfil constitucional fortalecido da Instituição", afirmou o ministro.

O ministro do Supremo validou as regras para uso dos recursos do fundo definidas no acordo. São elas:

o saldo corrigido será transferido para a conta única do Tesouro Nacional;
os recursos devem ser convertidos em receita da União, que se comprometerá a fazer com que passem a compor fonte de recursos específica, passível de acompanhamento pelos órgãos de controle;
foi publicada a portaria “Recursos Oriundos de Leis ou Acordos Anticorrupção” para assegurar transparência e controle quando os valores forem utilizados para financiar as despesas públicas do acordo;
a União, por meio do Ministério da Economia, se compromete a garantir as condições necessárias para execução total das despesas neste exercício financeiro e nos seguintes, até o completo exaurimento dos valores;
a realização das despesas será concretizada especialmente por meio da concessão de limite de empenho e de movimentação financeira (“descontingenciamento”) e da abertura de créditos adicionais;
os valores destinados aos Estados da região amazônica serão divididos mediante critérios a serem definidos pelos ministérios.

Acordo da Petrobras
Para encerrar investigações sobre a empresa nos Estados Unidos, a Petrobras acordou com autoridades norte-americanas o pagamento de US$ 853,2 milhões. Desse valor, US$ 682 milhões devem ser aplicados no Brasil - cerca de R$ 2,6 bilhões.

O valor é fruto de reparação de danos admitidos pela Petrobras, e não da devolução de valores com delações premiadas.

O montante foi depositado em uma conta judicial, mas está bloqueado.

Inicialmente, a força-tarefa da Lava Jato havia feito um acordo com a Petrobras para criar um fundo privado com parte dinheiro. Mas o ministro do STF, Alexandre de Moraes, suspendeu o ato.'''

### Pipeline de execução principal

###1) Cria tabela de frequência

In [ ]:
tabela_freq = tabela_de_frequencia(text)
print(tabela_freq)

{'O': 7, 'ministro': 8, 'Alexandre': 3, 'de': 28, 'Moraes': 5, ',': 35, 'do': 19, 'Supremo': 2, 'Tribunal': 1, 'Federal': 1, '(': 8, 'STF': 2, ')': 10, 'validou': 3, 'nesta': 1, 'terça-feira': 1, '17': 1, 'o': 13, 'acordo': 9, 'para': 22, 'utilização': 2, 'fundo': 4, 'bilionário': 1, 'da': 28, 'Petrobras': 6, '–': 1, 'cerca': 2, 'R': 11, '$': 13, '2,6': 2, 'bilhões': 2, '.': 18, 'foi': 3, 'assinado': 1, 'pelo': 2, 'governo': 1, 'federal': 1, 'representantes': 1, 'Câmara': 1, 'e': 19, 'Senado': 1, 'Procuradoria': 1, 'Geral': 1, 'República': 1, 'PGR': 1, 'enviado': 1, 'homologação': 2, 'a': 24, 'seguinte': 1, 'divisão': 1, 'uso': 3, ':': 5, '1': 2, '1,601': 1, 'bilhão': 3, 'educação': 2, 'sendo': 2, 'ações': 5, 'relacionadas': 2, 'à': 1, 'infantil': 1, ';': 12, '250': 2, 'milhões': 8, 'Ministério': 5, 'Cidadania': 1, 'ao': 4, 'Programa': 1, 'Criança': 1, 'Feliz': 1, 'desenvolvimento': 1, 'integral': 1, 'Primeira': 1, 'Infância': 1, 'Ciência': 1, 'Tecnologia': 1, 'Inovações': 1, 'Comunica

###2) Segmenta as sentenças

In [ ]:
sentencas = sent_tokenize(text)
print(sentencas)

['O ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF), validou nesta terça-feira (17) o acordo para utilização do fundo bilionário da Petrobras – de cerca de R$ 2,6 bilhões.', 'O acordo foi assinado pelo governo federal, representantes de Câmara e Senado e da Procuradoria Geral da República (PGR) e enviado para homologação pelo ministro.', 'Moraes validou a seguinte divisão para uso do fundo:\n\n1) R$ 1,601 bilhão para educação, sendo:\n\nR$ 1 bilhão para ações relacionadas à educação infantil;\nR$ 250 milhões para o Ministério da Cidadania, para ações relacionadas ao Programa Criança Feliz (desenvolvimento integral da Primeira Infância);\nR$ 250 milhões para o Ministério da Ciência, Tecnologia, Inovações e Comunicações, para ações de empreendedorismo e bolsas do Conselho Nacional de Desenvolvimento Científico e Tecnológico (CNPq);\nR$ 100 milhões para ações socioeducativas em cooperação com os Estados, preferencialmente por intermédio do Ministério da Mulher, da Família 

### 3) Pontuação das sentenças

In [ ]:
pontuacao_sentencas = pontuar_sentencas(sentencas, tabela_freq)
print(pontuacao_sentencas)

{'O ministro': 10.43245160636465, 'O acordo f': 10.48, 'Moraes val': 6.189655172413793, '2) R$ 1,06': 7.195652173913044, 'Na decisão': 7.2894736842105265, 'Com relaçã': 6.387755102040816, 'Alexandre ': 9.5, '"A eventua': 5.6, 'São elas:\n': 3.7981651376146788, 'Acordo da ': 8.441176470588236, 'Desse valo': 8.384615384615385, 'O valor é ': 8.733333333333333, 'O montante': 9.095238095238095, 'Inicialmen': 9.727272727272727, 'Mas o mini': 14.166666666666666}


###4) Definição de threshold

In [ ]:
# Você pode mudar os valores de acordo com o método de sumarização que for utilizar/desenvolver

#Threshold de pontuação
threshold = calcula_pontuacao_media(pontuacao_sentencas)

#Threshold de número de sentenças
#threshold = 3

#Threshold de taxa compressão
#threshold = 0.3

In [ ]:
threshold
# Se uma sentença tiver pontuação acima do threshold, ela erá escolhida (extraída) para o texto final
# Para abordagem de sumarizaçãio extrativa, não gera texto novo.

8.361430370284797

###5) Gera o sumário
Alterne entre as 3 chamadas de função abaixo para modificar a método de sumarização.
Desenvolva duas novas versões do algoritmo, uma para criar um sumário final com número fixo de sentenças e outra para criar baseado em uma taxa de compressão (e.g., sumário terá 30% do tamanho original do texto).

O gabarito dos métodos pedidos pode ser encontrado [aqui](https://colab.research.google.com/drive/10nF0N18xeipjSJ-1PzPDdITTqIr1akPi).

In [ ]:
#Threshold de pontuação
sumario = gerar_sumario(sentencas, pontuacao_sentencas, threshold)

#Threshold de número de sentenças
#sumario = gerar_sumario_fixo(sentencas, pontuacao_sentencas, threshold)

#Threshold de taxa compressão
#sumario = gerar_sumario_compressao(sentencas, pontuacao_sentencas, threshold)
print(sumario)

 O ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF), validou nesta terça-feira (17) o acordo para utilização do fundo bilionário da Petrobras – de cerca de R$ 2,6 bilhões. O acordo foi assinado pelo governo federal, representantes de Câmara e Senado e da Procuradoria Geral da República (PGR) e enviado para homologação pelo ministro. O ministro também concedeu pedido da Caixa para que a instituição desconte a remuneração relativa ao tempo em que permaneceu com a guarda dos valores. Alexandre de Moraes declarou ainda a nulidade do acordo firmado entre a força-tarefa da Lava Jato e a Petrobras. O ministro já havia suspenso a eficácia do acordo em março. O ministro do Supremo validou as regras para uso dos recursos do fundo definidas no acordo. Acordo da Petrobras
Para encerrar investigações sobre a empresa nos Estados Unidos, a Petrobras acordou com autoridades norte-americanas o pagamento de US$ 853,2 milhões. Desse valor, US$ 682 milhões devem ser aplicados no Brasil - ce

## Atividades

<b>1. Execute a sumarização do texto 1 e analise a tabela de frequência. Você considera que os tokens com maior peso realmente são os mais importantes?</b>



In [ ]:
# Texto 1
text1 = """Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa.

Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero.

O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”.

Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México).

Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos. O ataque deixou seis mortos.

Os confessores, identificados como Particio Reyes, Jonathan Osorio e Agustín García Reyes, dizem que receberam os 43 estudantes no lixão de Cocula, a 22 km de Iguala. Segundo os pistoleiros, 15 deles chegaram ao local mortos com sinais de asfixia.

Segundo a Procuradoria-Geral do México, os detidos não disseram quem levou os estudantes e quem era o mandante da emboscada.

O órgão, porém, acredita que o mandante foi o prefeito de Iguala, Jose Luis Abarca, preso na quarta (5).

A intenção seria evitar que os alunos atrapalhassem um evento em que sua mulher, María de los Ángeles Pineda, seria lançada como candidata a sucedê-lo. A mulher de Abarca é irmã de três chefes do Guerreros Unidos.

FAMILIARES

Em entrevista, os parentes disseram não acreditar na versão do procurador-geral e pediram que o material recolhido seja analisado por peritos independentes.

Para eles, o governo quer fazer com que eles acreditem que seus filhos estão mortos. “Sequer mostraram fotos dos nossos filhos. Enquanto não houver provas, nossos filhos estão vivos”, disse Felipe de la Cruz, pai de um dos alunos.

Os pais pediram ao governo que prossiga com as buscas e permita a assistência técnica da Comissão Interamericana de Direitos Humanos."""


In [ ]:
# Cria tabela de frequencia do texto 1
tabela_freq = tabela_de_frequencia(text1)
print(tabela_freq)

{'Dezenas': 1, 'de': 19, 'veículos': 1, 'foram': 2, 'incendiados': 1, 'em': 6, 'frente': 1, 'a': 9, 'sede': 1, 'do': 9, 'governo': 3, 'da': 7, 'região': 1, 'Guerrero': 2, ',': 26, 'no': 2, 'México': 3, 'um': 3, 'protesto': 2, 'pelo': 1, 'desaparecimento': 1, 'e': 7, 'morte': 1, '43': 2, 'estudantes': 4, 'escola': 2, 'normal': 1, 'rural': 1, 'Ayotzinapa': 1, '.': 18, 'Mais': 1, '300': 1, 'jovens': 2, 'maioria': 1, 'com': 5, 'o': 7, 'rosto': 1, 'coberto': 1, 'atacaram': 1, 'fachada': 1, 'edifício': 1, 'Chipancingo': 1, 'capital': 1, 'O': 4, 'ocorreu': 1, 'após': 1, 'procurador-geral': 2, 'República': 1, 'país': 1, 'Jesús': 1, 'Murillo': 1, 'Karam': 1, 'informar': 1, 'que': 10, 'três': 2, 'homens': 1, 'suspeitos': 1, 'ser': 1, 'integrantes': 1, 'cartel': 1, 'Guerreros': 3, 'Unidos': 3, 'confessaram': 1, 'ter': 1, 'matado': 1, 'os': 9, 'queimado': 1, 'seus': 2, 'corpos': 1, 'presidente': 1, 'Enrique': 1, 'Peña': 1, 'Nieto': 1, 'prometeu': 1, 'na': 3, 'sexta': 1, '(': 3, '7': 1, ')': 3, 'pu

RESPOSTA: os tokens com maior peso NÃO são os mais importantes, ref. a saída 1.

In [ ]:
# Segmenta as sentenças
sentencas = sent_tokenize(text1)
print(sentencas)

['Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa.', 'Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero.', 'O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos.', 'O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”.', 'Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México).', 'Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos.', 'O ataque deixou seis mortos.', 'Os confessores, identificados como

In [ ]:
# Calcula a pontuação das sentenças
pontuacao_sentencas = pontuar_sentencas(sentencas, tabela_freq)
print(pontuacao_sentencas)

{'Dezenas de': 4.588235294117647, 'Mais de 30': 6.6, 'O protesto': 4.838709677419355, 'O presiden': 4.423076923076923, 'Os jovens ': 5.178571428571429, 'Na saída d': 5.555555555555555, 'O ataque d': 7.7272727272727275, 'Os confess': 5.814814814814815, 'Segundo os': 5.818181818181818, 'Segundo a ': 6.208333333333333, 'O órgão, p': 5.625, 'A intenção': 5.033333333333333, 'A mulher d': 6.6923076923076925, 'FAMILIARES': 4.806451612903226, 'Para eles,': 5.9, '“Sequer mo': 5.533333333333333, 'Enquanto n': 5.541666666666667, 'Os pais pe': 4.5}


In [ ]:
# Define o threshold
#Threshold de pontuação
threshold = calcula_pontuacao_media(pontuacao_sentencas)

#Threshold de número de sentenças
#threshold = 3

#Threshold de taxa compressão
#threshold = 0.3

# Ajuste do threshold
# Aumentar em 50%
threshold_aumentado = threshold * 1.5

# Diminuir em 50%
threshold_diminuido = threshold * 0.5

# Aumentar em 75%
threshold_aumentado_75 = threshold * 1.75

# Diminuir em 75%
threshold_diminuido_75 = threshold * 0.25

# Aumentar em 100%
threshold_aumentado_100 = threshold * 2.0

# Diminuir em 100%
threshold_diminuido_100 = threshold * 0.0

#threshold # usado na 1a execucao pra mostrar o valor

In [ ]:
# Gera o sumário
#Threshold de pontuação
sumario = gerar_sumario(sentencas, pontuacao_sentencas, threshold)

print(sumario)

 Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O ataque deixou seis mortos. Os confessores, identificados como Particio Reyes, Jonathan Osorio e Agustín García Reyes, dizem que receberam os 43 estudantes no lixão de Cocula, a 22 km de Iguala. Segundo os pistoleiros, 15 deles chegaram ao local mortos com sinais de asfixia. Segundo a Procuradoria-Geral do México, os detidos não disseram quem levou os estudantes e quem era o mandante da emboscada. O órgão, porém, acredita que o mandante foi o prefeito de Iguala, Jose Luis Abarca, preso na quarta (5). A mulher de Abarca é irmã de três chefes do Guerreros Unidos. Para eles, o governo quer fazer com que eles acreditem que seus filhos estão mortos.


<b>Após isso, descomente as linhas abaixo da função tabela_de_frequencia(), e re-execute.</b>

```
#word = stemmer.stem(word)
#if word in stopWords:
#    continue
```

<b>O que aconteceu?</b>





RESPOSTA: Não criou a tabela de frequencia, não calculou a pontuação nem o threshold, não fez o sumário.

<b>2. Verifique os resultados da sumarização ao alterarmos o valor do threshold (e.g., aumentar e diminuir em 50%, 75%, 100%). Após análise responda: o que você acha do uso de um threshold baseado em uma pontuação? Funcionaria bem para todos casos de sumarização?</b>


In [ ]:
# Gera os sumários com os thresholds ajustados
sumario_threshold_aumentado = gerar_sumario(sentencas, pontuacao_sentencas, threshold_aumentado)
sumario_threshold_diminuido = gerar_sumario(sentencas, pontuacao_sentencas, threshold_diminuido)
sumario_threshold_aumentado_75 = gerar_sumario(sentencas, pontuacao_sentencas, threshold_aumentado_75)
sumario_threshold_diminuido_75 = gerar_sumario(sentencas, pontuacao_sentencas, threshold_diminuido_75)
sumario_threshold_aumentado_100 = gerar_sumario(sentencas, pontuacao_sentencas, threshold_aumentado_100)
sumario_threshold_diminuido_100 = gerar_sumario(sentencas, pontuacao_sentencas, threshold_diminuido_100)

In [ ]:
# Imprime os sumários gerados
print("1-Sumário com threshold aumentado em 50%:")
print(sumario_threshold_aumentado)

print("\n2-Sumário com threshold diminuído em 50%:")
print(sumario_threshold_diminuido)

print("\n3-Sumário com threshold aumentado em 75%:")
print(sumario_threshold_aumentado_75)

print("\n4-Sumário com threshold diminuído em 75%:")
print(sumario_threshold_diminuido_75)

print("\n5-Sumário com threshold aumentado em 100%:")
print(sumario_threshold_aumentado_100)

print("\n6-Sumário com threshold diminuído em 100%:")
print(sumario_threshold_diminuido_100)

1-Sumário com threshold aumentado em 50%:


2-Sumário com threshold diminuído em 50%:
 Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa. Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”. Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México). Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos.

RESPOSTA: O threshold baseado em pontuação não produziu a sumarização quando foi aumentado, por isso, não funciona em todos os casos de sumarização.

<b>3. Na implementação do algoritmo em que o tamanho do sumário final é definido por um threshold que estabeleçe um número fixo de sentenças. Quais benefícios e malefícios desta opção?</b>


In [ ]:
# Define o threshold
#Threshold de pontuação
threshold = calcula_pontuacao_media(pontuacao_sentencas)

#Threshold de número de sentenças
threshold = 3

In [ ]:
# Gera o sumário
#Threshold de pontuação
sumario = gerar_sumario(sentencas, pontuacao_sentencas, threshold)

print(sumario)

 Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa. Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”. Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México). Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos. O ataque deixou seis mortos. Os confessores, identificados como Particio Reyes, Jonat

<p align='justify'>RESPOSTA: <b>A saída com nro fixo de sentenças é maior do que a saída sem fixar</b> (Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa. Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”. Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México). Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos. O ataque deixou seis mortos. Os confessores, identificados como Particio Reyes, Jonathan Osorio e Agustín García Reyes, dizem que receberam os 43 estudantes no lixão de Cocula, a 22 km de Iguala. Segundo os pistoleiros, 15 deles chegaram ao local mortos com sinais de asfixia. Segundo a Procuradoria-Geral do México, os detidos não disseram quem levou os estudantes e quem era o mandante da emboscada. O órgão, porém, acredita que o mandante foi o prefeito de Iguala, Jose Luis Abarca, preso na quarta (5). A intenção seria evitar que os alunos atrapalhassem um evento em que sua mulher, María de los Ángeles Pineda, seria lançada como candidata a sucedê-lo. A mulher de Abarca é irmã de três chefes do Guerreros Unidos. FAMILIARES</p>

<p align='justify'>Em entrevista, os parentes disseram não acreditar na versão do procurador-geral e pediram que o material recolhido seja analisado por peritos independentes. Para eles, o governo quer fazer com que eles acreditem que seus filhos estão mortos. “Sequer mostraram fotos dos nossos filhos. Enquanto não houver provas, nossos filhos estão vivos”, disse Felipe de la Cruz, pai de um dos alunos. Os pais pediram ao governo que prossiga com as buscas e permita a assistência técnica da Comissão Interamericana de Direitos Humanos.)</p>

- A saída do sumário sem fixar nro de sentenças é esta: Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O ataque deixou seis mortos. Os confessores, identificados como Particio Reyes, Jonathan Osorio e Agustín García Reyes, dizem que receberam os 43 estudantes no lixão de Cocula, a 22 km de Iguala. Segundo os pistoleiros, 15 deles chegaram ao local mortos com sinais de asfixia. Segundo a Procuradoria-Geral do México, os detidos não disseram quem levou os estudantes e quem era o mandante da emboscada. O órgão, porém, acredita que o mandante foi o prefeito de Iguala, Jose Luis Abarca, preso na quarta (5). A mulher de Abarca é irmã de três chefes do Guerreros Unidos. Para eles, o governo quer fazer com que eles acreditem que seus filhos estão mortos.

<b>4. Na implementação do algoritmo em que o tamanho do sumário final é definido por um threshold que estabeleçe que o sumário final deve ter no máximo 10% do original. Quais benefícios e malefícios desta opção?</b>



In [ ]:
# Define o threshold
#Threshold de pontuação
threshold = calcula_pontuacao_media(pontuacao_sentencas)

#Threshold de número de sentenças
#threshold = 3

#Threshold de taxa compressão
#threshold = 0.3

# Ajuste do threshold
# Diminuir em 10%
threshold_diminuido = threshold * 0.01

In [ ]:
# Gera o sumário com threshold menor
sumario_threshold_diminuido = gerar_sumario(sentencas, pontuacao_sentencas, threshold_diminuido)

In [ ]:
# Imprime o sumário gerado
print("\n2-Sumário com threshold diminuído em 10%:")
print(sumario_threshold_diminuido)


2-Sumário com threshold diminuído em 10%:
 Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa. Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”. Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México). Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos. O ataque deixou seis mortos. Os confessore

RESPOSTA: Não se percebe a diferença da diminuição.  
Saída do sumário diminuído em 10%: (Dezenas de veículos foram incendiados em frente a sede do governo da região de Guerrero, no México, em um protesto pelo desaparecimento e morte de 43 estudantes da escola normal rural de Ayotzinapa. Mais de 300 jovens, a maioria com o rosto coberto, atacaram a fachada do edifício em Chipancingo, capital de Guerrero. O protesto ocorreu após o procurador-geral da República do país, Jesús Murillo Karam, informar que três homens suspeitos de ser integrantes do cartel Guerreros Unidos confessaram ter matado os estudantes e queimado seus corpos. O presidente Enrique Peña Nieto prometeu na sexta (7) punir todos os responsáveis pelos “crimes abomináveis”. Os jovens sumiram em 26 de setembro, depois de arrecadar fundos para a escola em Iguala (a 192 km da Cidade do México). Na saída da cidade, dois ônibus que voltavam à instituição com os alunos foram alvejados por policiais e traficantes do Guerreros Unidos. O ataque deixou seis mortos. Os confessores, identificados como Particio Reyes, Jonathan Osorio e Agustín García Reyes, dizem que receberam os 43 estudantes no lixão de Cocula, a 22 km de Iguala. Segundo os pistoleiros, 15 deles chegaram ao local mortos com sinais de asfixia. Segundo a Procuradoria-Geral do México, os detidos não disseram quem levou os estudantes e quem era o mandante da emboscada. O órgão, porém, acredita que o mandante foi o prefeito de Iguala, Jose Luis Abarca, preso na quarta (5). A intenção seria evitar que os alunos atrapalhassem um evento em que sua mulher, María de los Ángeles Pineda, seria lançada como candidata a sucedê-lo. A mulher de Abarca é irmã de três chefes do Guerreros Unidos. FAMILIARES

Em entrevista, os parentes disseram não acreditar na versão do procurador-geral e pediram que o material recolhido seja analisado por peritos independentes. Para eles, o governo quer fazer com que eles acreditem que seus filhos estão mortos. “Sequer mostraram fotos dos nossos filhos. Enquanto não houver provas, nossos filhos estão vivos”, disse Felipe de la Cruz, pai de um dos alunos. Os pais pediram ao governo que prossiga com as buscas e permita a assistência técnica da Comissão Interamericana de Direitos Humanos.)

## Desafios


*   Implementar função de filtragem das sentenças, para retirar sentenças com alto grau de similaridade semântica (pode-se fazer uso de WordNets, Word Embeddings, etc.). Antes de inserir uma nova sentença na lista de selecionadas, verificar similaridade da mesma com as previamente selecionadas. Caso tenha alto grau de similaridade (defina um threshold), descartar.


*   Faça uma implementação que ao invés de usar apenas a frequência simples como fator de importância, que utilize o TF-IDF (https://github.com/akashp1712/nlp-akash/blob/master/text-summarization/TF_IDF_Summarization.py)

*   Faça um algoritmo que receba uma URL de algum site de notícias, faça o web-scrapping dos dados, e sumarize a notícia.

## Referências e Material complementar

*   [Seminário - Sumarização de Textos - Prof. Lucas Oliveira](https://drive.google.com/file/d/1RFaXe_TIdcfgp5N3b5aSQocuDKdy-uvi/view?usp=sharing)
*   [Text summarization in 5 steps using NLTK: WordFrequency Algorithm](https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65)
*   [Utilizando processamento de linguagem natural para criar um sumarização automática de textos](https://medium.com/@viniljf/utilizando-processamento-de-linguagem-natural-para-criar-um-sumariza%C3%A7%C3%A3o-autom%C3%A1tica-de-textos-775cb428c84e)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500) e editado por Carla Silveira.